## Graph API

In [13]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import  StateGraph, START, END
from langgraph.graph.message import add_messages ## work add the msg appeneded to the state

In [21]:
## state
class State(TypedDict):
    # Msg have type list defines how this state key should be updated

    messages : Annotated[list, add_messages] ## appended to the list

graph_bilder =StateGraph(State)

graph_bilder

In [24]:
## Building the graph

import os
from dotenv import load_dotenv
load_dotenv()

False